In [ ]:
from google.colab import drive, output
drive.mount('/content/drive')
import sys
%cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
#sys.path.append('/content/drive/MyDrive/Deep Learning/AutoEncoders/Project/VQVAE_Working/data')
#sys.path.append('/content/drive/MyDrive/Deep Learning/AutoEncoders/Project/VQVAE_Working/models')
sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Dataloader')
sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Models')
sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Modules')
%load_ext autoreload
%autoreload 1
!pip install torchaudio
!pip install wandb --upgrade
!wandb login
output.clear()

In [ ]:
from __future__ import print_function
import os
import sys
import matplotlib.pyplot as plt
import numpy as np

from six.moves import xrange

#import umap
import wandb
import torch
torch.cuda.empty_cache()
import gc
gc.collect()
torch.cuda.empty_cache()

from scipy import signal
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim

from torch.utils.data import random_split
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid
import torch.nn.functional as F

#from ResidualStack import ResidualStack
#from Residual import Residual

from Jaguas_DataLoader import SoundscapeData
from Models import VAE

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = xm.xla_device()
print(device)

from datetime import timedelta
import wandb
from wandb import AlertLevel

wandb.login()

cuda


wandb: Currently logged in as: danielnieto. Use `wandb login --relogin` to force relogin


True

In [ ]:
root_path = '/content/drive/Shareddrives/ConservacionBiologicaIA/Datos/Jaguas_2018'


dataset = SoundscapeData(root_path, 1, "wav")
dataset_train, dataset_test = random_split(dataset,
                                           [round(len(dataset)*0.0250), len(dataset) - round(len(dataset)*0.0250)], 
                                           generator=torch.Generator().manual_seed(1024))

config = {
    "batch_size" : 8,
    "num_epochs": 5,
    "num_training_updates" : len(dataset_train),
    "num_hiddens" : 64,
    "embedding_dim" : 128,
    "zDim" : 128,
    "commitment_cost" : 0.25,
    "decay" : 0.99,
    "learning_rate" : 1e-3,
    "dataset": "Audios Jaguas",
    "architecture": "VQ-VAE",
}

training_loader = DataLoader(dataset_train, batch_size=config["batch_size"])
test_loader = DataLoader(dataset_test, batch_size=config["batch_size"])


model = VAE(num_hiddens=config["num_hiddens"],
              zDim=config["zDim"]).to(device)

optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"], amsgrad=False)


wandb.finish()
wandb.init(project="VAE-Jaguas", config=config)
wandb.watch(model)


[]

In [ ]:
def testModel(model, iterator):
    model.eval()
    (valid_originals, _,_) = next( iterator)
    valid_originals = torch.reshape(valid_originals, (valid_originals.shape[0] * valid_originals.shape[1], 
                                                  valid_originals.shape[2], valid_originals.shape[3]))
    valid_originals = torch.unsqueeze(valid_originals,1)

    valid_originals = valid_originals.to(device)

    mu, logvar = model.encoder(valid_originals)
    z = model.reparameterize(mu, logvar)
    valid_reconstructions = model.decoder(z)
    output =  torch.cat((valid_originals[0:8], valid_reconstructions[0:8]), 0)
    img_grid = make_grid(output,nrow= 8,pad_value=20)

    recon_error = F.mse_loss(valid_originals, valid_reconstructions)

    fig, ax = plt.subplots(figsize=(20,5))
    ax.imshow(img_grid[1,:,:].cpu(), vmin=0,vmax=1)
    ax.axis("off")
    plt.show()
    return fig, recon_error

In [ ]:
model.train()
train_res_recon_error = []
train_res_perplexity = []
iterator = iter(test_loader)
wandb.watch(model, F.mse_loss, log="all", log_freq=1)
error_files = []

for epoch in range(config["num_epochs"]):
  for i in xrange(config["num_training_updates"]):
      model.train()
      try:
         (data, _,_) = next(iter(training_loader))
      except:
        error_files.append[i]
        continue
          
      data = torch.reshape(data, (data.shape[0] * data.shape[1], data.shape[2], data.shape[3]))
      data = torch.unsqueeze(data,1)
      data = data.to(device)
      print(data.shape)
      optimizer.zero_grad()
      data_recon, mu, logvar = model(data)
      print(data_recon.shape)
      
      recon_error = F.mse_loss(data_recon, data) #/ data_variance
      loss = recon_error
      loss.backward()

      optimizer.step()

      print(f'epoch: {epoch} of {config["num_epochs"]} \t iteration: {(i+1)} of {config["num_training_updates"]} \t loss: {np.round(loss.item(),4)}')

      if (i+1) % 5 == 0:
        #torch.save(model.state_dict(),f'model_{epoch}_{i}.pkl')
        fig, test_error = testModel(model, iterator)
        images = wandb.Image(fig, caption= f"recon_error: {np.round(test_error.item(),4)}")
        wandb.log({"examples": images})

wandb.finish()

torch.Size([472, 1, 129, 513])
inputs shape torch.Size([472, 1, 129, 513])
conv1: torch.Size([472, 16, 126, 510])
conv2: torch.Size([472, 64, 62, 254])
conv3: torch.Size([472, 128, 30, 126])
x.view: torch.Size([472, 483840])


RuntimeError: ignored

In [ ]:
a = torch.rand(1062,64,30,30)
s = a.view(1062,64,-1)
print(s.shape)
t = s.view(1062,64,)
print(t.shape)